In [1]:
import torch
torch.cuda.is_available()

False

In [3]:
import torch
print(torch.rand(5, 3))

tensor([[0.2947, 0.6026, 0.3936],
        [0.1432, 0.2291, 0.5525],
        [0.5219, 0.0638, 0.4893],
        [0.5170, 0.8382, 0.2788],
        [0.0947, 0.8991, 0.6488]])
